<a href="https://colab.research.google.com/github/Sbarbagnem/AdvancedProject/blob/master/ProgettoAdvancedMachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Multi-label object classification

## Componenti del gruppo
* Carta Costantino
* Hamrani Hamza
* Ventura Samuele 793060

## Module Import


In [0]:
from google.colab import drive
import xml.etree.ElementTree as ET
from pathlib import Path
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import time
import os
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss, accuracy_score
import numpy as np
from ast import literal_eval
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, AveragePooling2D
from keras.optimizers import SGD
from math import ceil
from keras.callbacks import EarlyStopping, Callback
from keras import backend as K
from itertools import tee  # finally! I found something useful for it
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report
from keras.applications.vgg16 import preprocess_input, VGG16 
import tensorflow as tf
from collections import Counter
import matplotlib.pyplot as plt
from keras import layers, models, optimizers

Using TensorFlow backend.


## Costants

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
PATH_ABSOLUTE = '/content/drive/My Drive/Laurea Magistrale/Adavanched Machine Learning/Progetto/Dataset/'

PATH_TRAIN = PATH_ABSOLUTE + 'Train/'
PATH_TEST = PATH_ABSOLUTE + 'Test/'

PATH_ANNOTATIONS_TRAIN =  PATH_TRAIN + 'Annotations'
PATH_IMAGES_TRAIN = PATH_TRAIN + 'Images'

PATH_MAIN_VAL = PATH_TRAIN + 'Main/val.txt'

PATH_ANNOTATIONS_TEST =  PATH_TEST + 'Annotations'
PATH_IMAGES_TEST = PATH_TEST + 'Images'

PATH_CSV_IMAGE_LABEL_TRAIN_VAL = PATH_ABSOLUTE + 'image_label_train_val.csv'
PATH_CSV_IMAGE_LABEL_TRAIN = PATH_ABSOLUTE + 'image_label_train.csv'
PATH_CSV_IMAGE_LABEL_VAL = PATH_ABSOLUTE + 'image_label_val.csv'
PATH_CSV_IMAGE_LABEL_TEST = PATH_ABSOLUTE + 'image_label_test.csv'

## Util function

In [0]:
def xml_to_labels(xml_data, unique_labels):
  root = ET.XML(xml_data)
  labels = set() if unique_labels else []
  labels_add = labels.add if unique_labels else labels.append 
  for i, child in enumerate(root):
    if child.tag == 'filename':
      img_filename = child.text
    if child.tag == 'object':
      for subchild in child:
        if subchild.tag == 'name':
          labels_add(subchild.text)
  return img_filename, list(labels)

def get_labels(annotations_dir, unique_labels=True):
  '''
  # annotation_dir: directory with annotations file
  # unique_labels:  if true in presence of mutliple label of the same object we 
                    take label only one time
  Return all image-labels from annotations file in annotation_dir
  '''
  dir_path = Path(annotations_dir)
  for annotation_file in dir_path.iterdir():
    with open(annotation_file) as f:
      yield xml_to_labels(f.read(), unique_labels)
      
def plot_history(network_history, n_epochs):
  
    x_plot = list(range(1,n_epochs+1))
    
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(x_plot, network_history.history['loss'])
    plt.plot(x_plot, network_history.history['val_loss'])
    plt.legend(['Training', 'Validation'])

    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(x_plot, network_history.history['f1'])
    plt.plot(x_plot, network_history.history['val_f1'])
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()

## Creazione dataset csv (da eseguire solo la prima volta)
Crea i vari csv contenenti i dati di train, val e test da usare come dataframe nei generator


In [0]:
# Read annotatons file in train/annotations in dataframe 
# and save to csv image_label_train_val
df = pd.DataFrame(get_labels(PATH_ANNOTATIONS_TRAIN), columns=['filename', 'labels'])
df.to_csv(PATH_CSV_IMAGE_LABEL_TRAIN_VAL)

### Train data

In [0]:
# dal csv image_label_train_val tolgo quelle immagini da usare come validation,
# indicate nel file main/val.txt
df = pd.read_csv(PATH_CSV_IMAGE_LABEL_TRAIN_VAL, index_col=0)

print(df.shape)

lines = []

# image to use as validation
with open(PATH_MAIN_VAL) as file:
    lines = [line.strip() for line in file]
lines = [line + ".jpg" for line in lines]

train = df[~df['filename'].isin(lines)]

print(train.shape)

train.to_csv(PATH_CSV_IMAGE_LABEL_TRAIN)

(11302, 2)


### Validation data

In [0]:
# dal csv image_label_train_val tengo quelle immagini
# indicate nel file main/val.txt
val = df[df['filename'].isin(lines)]

print(val.shape)

val.to_csv(PATH_CSV_IMAGE_LABEL_VAL)

### Test data

In [0]:
# tutte le immagini in dataset/test/annotations
df = pd.DataFrame(get_labels(PATH_ANNOTATIONS_TEST), columns=['filename', 'labels'])
df.to_csv(PATH_CSV_IMAGE_LABEL_TEST)

## Salvo immagini in csv

In [0]:
# leggo da csv precedente e salvo train

# leggo da csv precedente e salvo val

## Image generator

In [0]:
import random

def random_sample_from_dataset(dataframe, path_images):

  image_generator = ImageDataGenerator( rescale=1/255,
                                        )

  '''images = image_generator.flow_from_dataframe(
      dataframe,
      directory=Path(path_images),
      x_col='filename',
      y_col='labels',
      target_size=((256, 256)),
      batch_size=64
  )'''

  idx_random = random.sample(range(0, len(dataframe)), 1000)

  # tengo solo righe df con idx_random

  # flow_from_dataframe da questo nuovo df

  # fit per estrarre media e deviazione standard

  # ritorno media e deviazione standard

  return idx_random

#idx_random = random_sample_from_dataset(train, PATH_IMAGES_TRAIN)

# The helper function
def multilabel_flow_from_dataframe(data_generator, mlb, df):
    for x, y in data_generator:
        assert isinstance(mlb, MultiLabelBinarizer), \
               "MultiLabelBinarizer is required."
        indices = y.astype(np.int).tolist()
        y_multi = mlb.transform(
             df.iloc[indices]['labels'].values
        )
        yield x, y_multi

BATCH_SIZE = 64

# usiamo 0.8 come train e 0.2 come val
train = pd.read_csv(PATH_CSV_IMAGE_LABEL_TRAIN, index_col=0)
train['labels'] = train['labels'].apply(lambda x: literal_eval(x))

# sarebbe il validation da usare come test, così manteniamo un equilibrio nelle classi
val = pd.read_csv(PATH_CSV_IMAGE_LABEL_VAL, index_col=0)
val['labels'] = val['labels'].apply(lambda x: literal_eval(x))

# add index to multilabel generator
train['index'] = np.arange(0,len(train))
val['index'] = np.arange(0,len(val))

'''
labels_count = Counter(label for lbs in train['labels'] for label in lbs)
total_count = sum(labels_count.values())
class_weights = {cls: total_count / count for cls, count in labels_count.items()}
plt.bar(class_weights.keys(), class_weights.values())

new = {}

for i,label in zip(range(0,20), class_weights.keys()):
  print(i, " ", label)
  new[i] = class_weights[label]

class_weights = new
'''
# per usare data augmentation le immagini devono essere in flow separati e nel nostro
# caso train e val sono insieme (se uso validation_split) per non fare data augmentation
# sul validation uso tutto train e val separatamente.
img_gen_train = ImageDataGenerator( rescale=1/255,
                                    #featurewise_center=True,
                                    #featurewise_std_normalization=True
                                  )
#img_gen_val = ImageDataGenerator(rescale=1/255, validation_split=0.5)

train_generator = img_gen_train.flow_from_dataframe(
    train,
    shuffle=True,
    directory=Path(PATH_IMAGES_TRAIN),
    x_col='filename',
    y_col='index',
    class_mode='other',
    target_size=((256, 256)),
    batch_size=BATCH_SIZE
)

val_generator = img_gen_train.flow_from_dataframe(
    val,
    shuffle=False,
    directory=Path(PATH_IMAGES_TRAIN),
    x_col='filename',
    y_col='index',
    class_mode='other',
    target_size=((256, 256)),
    batch_size=BATCH_SIZE,
    subset = 'training'
)

mlb = MultiLabelBinarizer()
mlb.fit(train['labels'].values)
mlb.classes_

train_gen_multi = multilabel_flow_from_dataframe(train_generator, mlb, train)
val_gen_multi = multilabel_flow_from_dataframe(val_generator, mlb, val)

'''
val_generator = img_gen_val.flow_from_dataframe(
    val,
    shuffle=False,
    directory=Path(PATH_IMAGES_TRAIN),
    x_col='filename',
    y_col='labels',
    class_mode='categorical',
    target_size=((512, 512)),
    batch_size=BATCH_SIZE,
    subset = 'training'
)

test_generator = img_gen_val.flow_from_dataframe(
    val,
    shuffle=False,
    directory=Path(PATH_IMAGES_TRAIN),
    x_col='filename',
    y_col='labels',
    class_mode='categorical',
    target_size=((512, 512)),
    batch_size=BATCH_SIZE,
    subset = 'validation'
)
'''
'''
from skimage import io

def imshow(image_RGB):
  io.imshow(image_RGB)
  io.show()

x,y = train_generator.next()

for i in range(0,11):
    image = x[i]
    imshow(image)
'''

TRAIN_LEN = train_generator.n
VAL_LEN = val_generator.n
#TEST_LEN = test_generator.n

,filename,labels
1,2010_005064.jpg,[dog]
2,2007_000713.jpg,[boat]
3,2010_003415.jpg,"[person, car]"
4,2011_006850.jpg,[person]
7,2008_005636.jpg,[cat]


## Modello base

### Metriche su validation completo

In [0]:
class Metrics(Callback):
  def __init__(self, validation_generator, validation_steps, threshold=0.5):
    self.validation_generator = validation_generator
    self.validation_steps = validation_steps or len(validation_generator)
    self.threshold = threshold

  def on_train_begin(self, logs={}):
    self.val_f1_scores = []
    #self.val_hamming_scores = []
    #self.subset_accuracy = []
    self.val_recalls = []
    self.val_precisions = []

  def on_epoch_end(self, epoch, logs={}):
    #gen_1, gen_2 = tee(self.validation_generator)
    y_true = np.vstack(next(self.validation_generator)[1] for _ in range(self.validation_steps)).astype('int')
    y_pred = (self.model.predict_generator(self.validation_generator, steps=self.validation_steps) > self.threshold).astype('int')
    # avarge indica come le misure sulle diverse classi sono mediate per avere un singolo valore di output
    f1 = f1_score(y_true, y_pred, average='macro')
    #hamming = hamming_loss(y_true, y_pred)
    #accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    self.val_f1_scores.append(f1)
    #self.val_hamming_scores.append(hamming)
    #self.subset_accuracy.append(accuracy)
    self.val_recalls.append(recall)
    self.val_precisions.append(precision)
    print(f"- val_f1: {f1:.5f} - val_precision: {precision:.5f} - val_recall: {recall:.5f}")
    return

### Metriche on batch

In [0]:

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))  

## Focal loss

In [0]:
def KerasFocalLoss(target, input):
    
    gamma = 2.
    alpha = 1.

    # from prob to logits
    input = tf.cast(input, tf.float32)
    
    max_val = K.clip(-input, 0, 1)

    # calculate binary cross entropy between target and input(prediction)
    BCE_loss = tf.keras.backend.binary_crossentropy(target, input, from_logits=True)
    
    pt = tf.exp(-BCE_loss)
    F_loss = alpha * (1-pt)**gamma * BCE_loss

    return tf.keras.backend.mean(F_loss)

    '''
    loss = input - input * target + max_val + K.log(K.exp(-max_val) + K.exp(-input - max_val))
    invprobs = tf.log_sigmoid(-input * (target * 2.0 - 1.0))
    loss = K.exp(invprobs * gamma) * loss
    
    return K.mean(K.sum(loss, axis=1))
    '''

### Modello

In [0]:
#metrics = Metrics(val_generator, ceil(VAL_LEN/BATCH_SIZE))
#metrics = Metrics()
input_image = layers.Input(shape = (256,256,3))
x = layers.Conv2D(8, (3, 3), padding = 'same',activation='relu')(input_image)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(8, (3, 3), padding = 'same', strides = 2, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size = (2, 2))(x)

x = layers.Conv2D(16, (3, 3), padding = 'same',activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(16, (3, 3), padding = 'same', strides = 2, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size = (2, 2))(x)

x = layers.Conv2D(32, (3, 3), padding = 'same',activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(32, (3, 3), padding = 'same', strides = 2, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size = (2, 2))(x)

x = layers.Conv2D(64, (3, 3), padding = 'same',activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, (3, 3), padding = 'same', strides = 2, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size = (2, 2))(x)

x = layers.Flatten()(x)
x = layers.Dense(128,activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64,activation='relu')(x)
x = layers.Dropout(0.3)(x)

output = layers.Dense(20, name = 'dense1')(x)
output = layers.Activation('sigmoid',name='output_layer')(output)


model = models.Model(input_image,output)

model.compile(loss=KerasFocalLoss, optimizer=optimizers.Adam(lr = 0.0001), metrics=[f1])
model.summary()

early_stop = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

net_history = model.fit_generator(train_gen_multi, epochs=50, verbose=1,
                  validation_data = val_gen_multi,
                  steps_per_epoch = ceil(TRAIN_LEN/BATCH_SIZE),
                  validation_steps = ceil(VAL_LEN/BATCH_SIZE),
                  callbacks = [early_stop],
                  workers = 8,
                  max_queue_size=10,
                  use_multiprocessing=True
                  #class_weight=class_weights
              )

## Plot performance train epoch

In [0]:
plot_history(net_history, 50)

## Performance on test

In [0]:
y_pred = (model.predict_generator(test_generator, 
                                  workers = 8, 
                                  max_queue_size=10,
                                  use_multiprocessing=True) > 0.5).astype('int')
mlb = MultiLabelBinarizer()
y_true = mlb.fit_transform(test_generator.labels)

report = classification_report(y_true=y_true, y_pred=y_pred)

## Modello VGG16 e Vector Machine per ogni label


## Ottimizzazione
* Provare diverse loss:
  * Hamming-loss
  * Exact match-ratio (subset accuracy) in sklearn è accuracy_score
* Provare diverse strutture (almeno 2/3)
* Provare qualche ottimizzatore diverso (Adam e SGD)

## Prova image da txt

In [0]:
PATH = PATH_TRAIN 

PATH_MAIN = PATH + 'Main/'

PATH_IMAGES = PATH + 'Images/'

labels = ['filename', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 
          'car', 'cat', 'chair', 'cow', 'diningtable',
          'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa',
          'train', 'tvmonitor']

train = pd.DataFrame(columns=labels)
val = pd.DataFrame(columns=labels)

# read all id of train and val
rows_train = []
f = open(PATH_MAIN + 'train.txt', "r")
for line in f.readlines():
  rows_train.append(line.split('\n')[0] + '.jpg')

rows_val = []
f = open(PATH_MAIN + 'val.txt', "r")
for line in f.readlines():
  rows_val.append(line.split('\n')[0] + '.jpg')

# initialize all row of dataframe to 0
train = train.append(pd.DataFrame(rows_train, columns=['filename']), ignore_index=True)
val = val.append(pd.DataFrame(rows_val, columns=['filename']), ignore_index=True)

for col in train.columns:
    if col != 'filename':
      train[col].values[:] = 0

for col in val.columns:
    if col != 'filename':
      val[col].values[:] = 0

# read txt in main and set 1 if there is that class in image
# if is -1 set to 0
for filename in os.listdir(PATH_MAIN):

    if filename.endswith("_train.txt"): 

      label = filename.split('_train.txt')[0]
      f = open(PATH_MAIN + filename, "r")

      for line in f.readlines():
        image = line.split(' ')[0] + '.jpg'
        presence = line.split(' ')[-1].strip()
        if presence == '1':
          train.loc[train['filename'] == image, label] = int(presence)

    elif filename.endswith("_val.txt"):

      label = filename.split('_val.txt')[0]
      f = open(PATH_MAIN + filename, "r")

      for line in f.readlines():
        image = line.split(' ')[0] + '.jpg'
        presence = line.split(' ')[-1].strip()
        if presence == '1':
          val.loc[val['filename'] == image, label] = int(presence)

train.to_csv(PATH_ABSOLUTE + 'train.csv')
val.to_csv(PATH_ABSOLUTE + 'val.csv')

In [0]:
from sklearn.model_selection import train_test_split

train = pd.read_csv(PATH_ABSOLUTE + 'train.csv', index_col=0)
val = pd.read_csv(PATH_ABSOLUTE + 'val.csv', index_col=0)

train_val = pd.concat([train, val])

x_train, x_val, y_train, y_val = train_test_split(  train_val['filename'], train_val[labels[1:]], 
                                                    test_size=0.2, random_state=42)
train_only = pd.concat([x_train, y_train], axis=1)
val_only = pd.concat([x_val, y_val], axis=1)

In [0]:
# calcualate class weights
train_only.head()

# frequency of labels
freq = train_only.apply(pd.value_counts).iloc[1,1:]

min_count = freq.min()

# between 0 and 1
freq_balance = freq.apply(lambda x: min_count/x)

In [0]:
from keras.applications.mobilenet import preprocess_input as preprocess_input_mobilenet
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg16
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50

BATCH_SIZE = 64
IM_SIZE = (224, 224)

img_gen_train = ImageDataGenerator( rescale=1/255,
                                    preprocessing_function=preprocess_input_vgg16,
                                    featurewise_center=True,
                                    featurewise_std_normalization=True,
                                    rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    horizontal_flip=True
                                  )
img_gen_val = ImageDataGenerator( rescale=1/255,
                                  preprocessing_function=preprocess_input_vgg16,
                                  featurewise_center=True,
                                  featurewise_std_normalization=True
                                )
img_gen_train.mean = np.array([0.4589, 0.4355, 0.4032], dtype=np.float32).reshape((1,1,3)) 
img_gen_train.std = np.array([0.2239, 0.2186, 0.2206], dtype=np.float32).reshape((1,1,3))
img_gen_val.mean = np.array([0.4589, 0.4355, 0.4032], dtype=np.float32).reshape((1,1,3)) 
img_gen_val.std = np.array([0.2239, 0.2186, 0.2206], dtype=np.float32).reshape((1,1,3))

train_generator = img_gen_train.flow_from_dataframe(
    train_only,
    shuffle=True,
    directory=Path(PATH_IMAGES),
    x_col='filename',
    y_col=labels[1:],
    class_mode='other',
    target_size=IM_SIZE,
    batch_size=BATCH_SIZE,

)
'''
from skimage import io

def imshow(image_RGB):
  io.imshow(image_RGB)
  io.show()

x,y = train_generator.next()

for i in range(0,11):
    image = x[i]
    imshow(image)
'''
val_generator = img_gen_val.flow_from_dataframe(
    val_only,
    shuffle=False,
    directory=Path(PATH_IMAGES),
    x_col='filename',
    y_col=labels[1:],
    class_mode='other',
    target_size=IM_SIZE,
    batch_size=BATCH_SIZE
)
TRAIN_LEN = train_generator.n
VAL_LEN = val_generator.n

Found 9232 validated image filenames.
Found 2308 validated image filenames.


In [0]:
def macro_soft_f1(y, y_hat):
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost

def macro_f1(y, y_hat, thresh=0.5):
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [0]:
#!pip install extra_keras_metrics

from keras.applications import VGG16, ResNet50, VGG19
from keras.layers import GlobalAveragePooling2D
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import Adam
from sklearn.metrics import average_precision_score
#from extra_keras_metrics import 

base_model_resnet50 = ResNet50(weights = 'imagenet', input_shape=(224,224,3), include_top = False)

for layer in base_model_resnet50.layers:
    layer.trainable=False

x = base_model_resnet50.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(20, activation = 'sigmoid')(x)
model = Model(input = base_model_resnet50.input, output = predictions)

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[macro_f1])

# callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              #patience=2, min_lr=0.000001, verbose=1)
#metrics = Metrics(val_generator, ceil(VAL_LEN/BATCH_SIZE))
net_history = model.fit_generator(train_generator, epochs=50, verbose=1,
                  validation_data = val_generator,
                  steps_per_epoch = ceil(TRAIN_LEN/BATCH_SIZE),
                  validation_steps = ceil(VAL_LEN/BATCH_SIZE),
                  callbacks = [early_stop],
                  workers = 8,
                  use_multiprocessing=True,
                  class_weight = freq_balance
              )

In [0]:
import matplotlib.pyplot as plt
import matplotlib.style as style

def learning_curves(history):
    """Plot the learning curves of loss and macro f1 score 
    for the training and validation datasets.
    
    Args:
        history: history callback of fitting a tensorflow keras model 
    """
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    macro_f1 = history.history['macro_f1']
    val_macro_f1 = history.history['val_macro_f1']
    
    epochs = len(loss)

    style.use("bmh")
    plt.figure(figsize=(8, 8))

    plt.subplot(2, 1, 1)
    plt.plot(range(1, epochs+1), loss, label='Training Loss')
    plt.plot(range(1, epochs+1), val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')

    plt.subplot(2, 1, 2)
    plt.plot(range(1, epochs+1), macro_f1, label='Training Macro F1-score')
    plt.plot(range(1, epochs+1), val_macro_f1, label='Validation Macro F1-score')
    plt.legend(loc='lower right')
    plt.ylabel('Macro F1-score')
    plt.title('Training and Validation Macro F1-score')
    plt.xlabel('epoch')

    plt.show()

In [0]:
learning_curves(net_history)

## TODO
* calcolare pesi delle classi sull'intero dataset prima di splittare
* provare a implementare la average precision come metrica